# Гаси Светла

## Лабораториска вежба 3

![Тука треба да видите .gif анимација, ама штом го читате ова нешто не е во ред.](images/example.gif)

Разгледуваме игра која се игра самостојно. Правилата на играта се:
- Се игра на табла со димензии `N` по `N`.
- Има празни и полни кругчиња, по едно кругче за секое поле од таблата.
- Се игра исклучиво со притискање на кругчињата.
- Ако се притисне некое кругче:
    - Се менува од празно во полно, или пак обратно, од полно во празно.
    - Неговите соседи се менуваат од празно во полно, или пак обратно, од полно во празно.
    - Соседи на едно кругче се соседните кругчиња кои се наоѓаат над, под, лево и десно.
- Целта на играта е сите кругчиња да се празни, во најмал број чекори.

Следи интерактивна апликација за оваа игра.

In [1]:
from plotly import graph_objects as go

In [2]:
def expand_square(square):
    neighbour_squares = []
    x, y = square
    for x, y in [(x, y), (x+1, y), (x-1, y), (x, y+1), (x, y-1)]:
        if 0 <= x < N and 0 <= y < N:
            neighbour_squares.append((x, y))
    return neighbour_squares

In [3]:
def update_fig(trace, points, selector):
    clicked_square = points.xs[0], points.ys[0]
    for x, y in expand_square(clicked_square):
        TABLE[y][x] = 0 if TABLE[y][x] else 1
    symbols = ['circle' if n else 'circle-open' for row in TABLE for n in row]
    fig.data[0].marker['symbol'] = symbols

def create_fig():
    fig = go.FigureWidget()
    x = [x for y in range(N) for x in range(N)]
    y = [y for y in range(N) for x in range(N)]
    symbols = ['circle' if n else 'circle-open' for row in TABLE for n in row]
    fig.add_scatter(x=x, y=y, mode='markers', marker_size=48, 
                    marker_symbol=symbols, marker_color='LightSkyBlue',
                    marker_line_width=6, marker_line_color='MediumPurple')
    fig.data[0].on_click(update_fig)
    fig.update_xaxes(range=[-0.5, N - 0.5], dtick=1, title='x', side='top')
    fig.update_yaxes(range=[-0.5, N - 0.5], dtick=1, title='y', autorange='reversed')
    fig.update_layout(width=120*N, height=120*N, showlegend=False)
    return fig

N = 4
TABLE = [
    [0, 0, 1, 0],
    [0, 0, 1, 1],
    [1, 1, 0, 0],
    [0, 1, 0, 0],
]

fig = create_fig()
fig

FigureWidget({
    'data': [{'marker': {'color': 'LightSkyBlue',
                         'line': {'color': 'MediumPurple', 'width': 6},
                         'size': 48,
                         'symbol': [circle-open, circle-open, circle, circle-open,
                                    circle-open, circle-open, circle, circle,
                                    circle, circle, circle-open, circle-open,
                                    circle-open, circle, circle-open, circle-open]},
              'mode': 'markers',
              'type': 'scatter',
              'uid': 'd367580f-ee5c-4ea0-8b0e-db9b8e6ef58b',
              'x': [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3],
              'y': [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3]}],
    'layout': {'height': 480,
               'showlegend': False,
               'template': '...',
               'width': 480,
               'xaxis': {'dtick': 1, 'range': [-0.5, 3.5], 'side': 'top', 'title': {'text': 'x'}},
       

**Задача:** Напишете алгоритам кој ќе стигне до целта и на некој начин ќе ја прикаже постапката, чекор по чекор, како да се стигне до решението.

In [4]:
TABLE_GOAL = [
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
]
N = 4

def expand_square(square):
    neighbour_squares = []
    x, y = square
    for x, y in [(x, y), (x+1, y), (x-1, y), (x, y+1), (x, y-1)]:
        if 0 <= x < N and 0 <= y < N:
            neighbour_squares.append((x, y))
    return neighbour_squares
#TABLE[y][x] = 0 if TABLE[y][x] else 1

def toggle(x, y, TABLE):
    new_table = [list(row) for row in TABLE] #kreiraj kopija od TABLE kako list za da e mozhna promena
    for dx, dy in [(0, 0), (1, 0), (-1, 0), (0, 1), (0, -1)]: #dx i dy se state i site sosedi
        new_x, new_y = x + dx, y + dy
    if 0 <= new_x < N and 0 <= new_y < N:  #proveri dali se vo rangot N
        new_table[new_x][new_y] = 0 if new_table[new_x][new_y] else 1 #napraj toggle na vrednosta na dx i dy
    return tuple(map(tuple, new_table)) #vrati tuple od novata sostojba, tuple za da nemat promeni

goal_state = tuple([[0] * N for _ in range(N)]) #matrica od 0

from collections import deque

def bfs(start, goal):
    queue = deque([start])  # Queue for BFS
    visited = set([start])  # Set to track visited states
    parent_map = {start: None}  # To track how we reached each state
    
    while queue:
        current_state = queue.popleft()
        
        # If we reached the goal state, reconstruct the path and return it
        if current_state == goal:
            path = []
            while current_state is not None:
                path.append(current_state)
                current_state = parent_map[current_state]
            return path[::-1]  # Return path in correct order (start -> goal)
        
        # Explore neighbors by toggling each cell
        for x in range(N):
            for y in range(N):
                # Apply the toggle function at each (x, y) to get a new state
                new_state = toggle(x, y, current_state)
                if new_state not in visited:
                    visited.add(new_state)
                    queue.append(new_state)
                    parent_map[new_state] = current_state  # Track how we reached new_state
    
    return None  # If no path is found (shouldn't happen in a solvable game)
start_state = (
    (0, 0, 1, 0),
    (0, 0, 1, 1),
    (1, 1, 0, 0),
    (0, 1, 0, 0)
)

# Perform BFS to find the shortest path
path = bfs(start_state, goal_state)

# Print the path of states
if path:
    for state in path:
        for row in state:
            print(row)
        print()
else:
    print("No solution found.")


No solution found.


In [ ]:
def toggle_square(square, N):
    neighbour_squares = []
    x, y = square
    for x, y in [(x, y), (x+1, y), (x-1, y), (x, y+1), (x, y-1)]:
        if 0 <= x < N and 0 <= y < N:
            neighbour_squares.append((x, y))
    for x, y in neighbour_squares:
        TABLE[y][x] = 0 if TABLE[y][x] else 1
    return neighbour_squares
    
def bfs(initial_table, goal_table):
    queue = deque([(initial_table, [])])
    visited = set()
    visited.add(tuple(map(tuple, initial_table)))
    while queue:
        current_state, path = queue.popleft()
        
        if current_state == goal_state:
            return path
        for x, y in range(N):
            square = (x,y)
            toggle_square(square, N)
            
                

In [12]:
def toggle_square(square, N):
    neighbour_squares = []
    x, y = square
    for x, y in [(x, y), (x+1, y), (x-1, y), (x, y+1), (x, y-1)]:
        if 0 <= x < N and 0 <= y < N:
            neighbour_squares.append((x, y))
    for x, y in neighbour_squares:
        TABLE[y][x] = 0 if TABLE[y][x] else 1
    return neighbour_squares

#def bfs(initial_table, N):
 #   goal_state = [[0] * N for _ in range(N)]
def breadth_first_search(initial_table, goal_table, expand_square):
    visited = set()
    queue = deque([(initial_table), [])])
    visited.add(tuple(map(tuple, initial_table)))
    while queue:
        current_state, path = queue.popleft()
        if current_state == goal_table:
            return path
        

## Лабораториска вежба 4

**Задачи:** 

- Смислете дојава и искористете ја функцијата за дојава во А* алгоритам за решавање на овој проблем.
- Покажете резултати дека А*, со помош на вашата функција за дојава, работи побрзо од пребарувањето со униформна цена.

In [ ]:
import heapq
N = 4
num_cells = N*N #br na elementi
goal_state = tuple([0] * num_cells)#cel: matrica od 0 odnosno iskluceni svetla

def toggle(state, x, y):
    new_state = state[:] #kopija na current state za da ne se modificira originalnata
    for dx, dy in [(x, y), (x+1, y), (x-1, y), (x, y+1), (x, y-1)]:
        nx, ny = x + dx, y + dy
        if 0 <= nx < N and 0 <= ny < N:
            idx = nx * N + ny #pozicijata vo matricata koja e pretstavena sega so lista/torka
            new_state[idx] = 1 - new_state[idx]  # Toggle the light (1->0, 0->1)
    return new_state
    
def h(state):
    c = sum(state)/5
    return c
    
#def heuristic(state): #ova precenvit
 #   return sum(state) #heuristic ni e brojot na '1' vo matricata, celta e da nema '1'

goal_state = [0] * (N * N) #celta e matrica so 0 (izgaseni se site svetla)

initial_state = (
    (0, 0, 1, 0),
    (0, 0, 1, 1),
    (1, 1, 0, 0),
    (0, 1, 0, 0)
)

In [10]:
def a_star_search(initial_state, goal_state, heuristic):
    if initial_state == goal_state:
        print('целта е постигната')
        return
    expanded = set()
    queue = [((0, 0), [initial_state])]
    heapq.heapify(queue)
    while queue:
        weight_tupple, vertex_list = heapq.heappop(queue)
        current_a_star_weight, current_path_weight = weight_tupple
        vertex_to_expand = vertex_list[-1]
        if vertex_to_expand in expanded:
            continue

        frontier = list(set([q[-1][-1] for q in queue]))
        yield frontier, vertex_list, vertex_to_expand

        for neighbour in expand_state(vertex_to_expand):
            if neighbour not in expanded:
                new_weight = get_weight(vertex_to_expand, neighbour)
                heuristic_score = heuristic(neighbour, goal_vertex)
                path_weight = current_path_weight + new_weight
                a_star_weight = path_weight + alpha * heuristic_score
                heapq.heappush(queue, ((a_star_weight, path_weight), vertex_list + [neighbour]))
        expanded.add(vertex_to_expand)
    yield [], [], goal_vertex